In [12]:
import os
import numpy as np

import feets

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
fs = feets.FeatureSpace()

data_path = os.path.join(os.path.abspath(os.path.dirname(feets.tests.__file__)), "data")
lc_path = os.path.join(data_path, "lc_1.3444.614.B_R.npz")

# recreate the lightcurve
with np.load(lc_path) as npz:
    magnitude = npz['mag']
    time = npz['time']
    
magnitude, time

(array([-6.081, -6.041, -6.046, ..., -6.009, -5.985, -5.997]),
 array([ 48823.477419,  48823.487014,  48823.496759, ...,  51531.401331,
         51541.344537,  51546.325197]))

## FATS

## Lomb-Scargle Single

In [3]:
from ext_lomb_scargle_orig import LombScargle as FATSLombScargle
lscargle = FATSLombScargle(fs)
%time fats_ls = lscargle.fit(magnitude, time, 6.)
fats_ls

CPU times: user 1.95 s, sys: 37.7 ms, total: 1.99 s
Wall time: 1.99 s


{u'PeriodLS': 0.93697445905023935,
 u'Period_fit': 3.1076457440343504e-113,
 u'Psi_CS': 0.19179114069585729,
 u'Psi_eta': 0.58910268294132195}

## Fourier Single

In [4]:
from ext_fourier_components_orig import FourierComponents as FATSFourierComponents
fc = FATSFourierComponents(fs)
%time fats_fc = fc.fit(magnitude, time, 6.)
fats_fc

CPU times: user 4.58 s, sys: 155 ms, total: 4.73 s
Wall time: 4.74 s


{u'Freq1_harmonics_amplitude_0': 0.13437970328353993,
 u'Freq1_harmonics_amplitude_1': 0.081782104118858376,
 u'Freq1_harmonics_amplitude_2': 0.050870593434863394,
 u'Freq1_harmonics_amplitude_3': 0.026198886228252748,
 u'Freq1_harmonics_rel_phase_0': 0.0,
 u'Freq1_harmonics_rel_phase_1': 0.3589872505549615,
 u'Freq1_harmonics_rel_phase_2': 0.78373753802371693,
 u'Freq1_harmonics_rel_phase_3': 1.3148411065542653,
 u'Freq2_harmonics_amplitude_0': 0.018352699400037011,
 u'Freq2_harmonics_amplitude_1': 0.0034160005551770825,
 u'Freq2_harmonics_amplitude_2': 0.0066381176236686131,
 u'Freq2_harmonics_amplitude_3': 0.0046673565716580726,
 u'Freq2_harmonics_rel_phase_0': 0.0,
 u'Freq2_harmonics_rel_phase_1': 3.0847901594538767,
 u'Freq2_harmonics_rel_phase_2': 0.46900853382169272,
 u'Freq2_harmonics_rel_phase_3': 1.7759627063676815,
 u'Freq3_harmonics_amplitude_0': 0.016548089932575488,
 u'Freq3_harmonics_amplitude_1': 0.0018836266253220439,
 u'Freq3_harmonics_amplitude_2': 0.0066137737410486

----

## feets

## Lomb-Scargle Single

In [5]:
from feets.extractors.ext_lomb_scargle import LombScargle as FeetsLombScargle, lscargle as ls
lscargle = FeetsLombScargle(fs)
fasper_kwds = {"autopower_kwds": {"samples_per_peak": 6., "nyquist_factor": 10}}

params = {
    "lscargle_kwds": {
        "autopower_kwds": {
            "normalization": "standard",
            "samples_per_peak": 6., "nyquist_factor": 10
        }
    },
    "fap_kwds": {
        "normalization": "standard",
        "method": "simple"}
}


%time feets_ls = lscargle.fit(magnitude, time, **params)
feets_ls

CPU times: user 86.5 ms, sys: 3.92 ms, total: 90.4 ms
Wall time: 91 ms


{u'PeriodLS': 0.93694759085825552,
 u'Period_fit': 0.0,
 u'Psi_CS': 0.19014757424308459,
 u'Psi_eta': 0.66944304684644684}

## Fourier single

In [6]:
from feets.extractors.ext_fourier_components import FourierComponents as FeetsFourierComponents
fc = FeetsFourierComponents(fs)
fasper_kwds = {"autopower_kwds": {"samples_per_peak": 6., "nyquist_factor": 100}}
%time feets_fc = fc.fit(magnitude, time, lscargle_kwds=fasper_kwds)
feets_fc

CPU times: user 1.59 s, sys: 124 ms, total: 1.72 s
Wall time: 1.72 s


{u'Freq1_harmonics_amplitude_0': 0.13343381203485169,
 u'Freq1_harmonics_amplitude_1': 0.078404211897825205,
 u'Freq1_harmonics_amplitude_2': 0.050566009236220351,
 u'Freq1_harmonics_amplitude_3': 0.026084952269435833,
 u'Freq1_harmonics_rel_phase_0': 0.0,
 u'Freq1_harmonics_rel_phase_1': 0.15681150010700362,
 u'Freq1_harmonics_rel_phase_2': 0.40575216196173691,
 u'Freq1_harmonics_rel_phase_3': 0.66531282827202887,
 u'Freq2_harmonics_amplitude_0': 0.016620167734147397,
 u'Freq2_harmonics_amplitude_1': 0.0033078430295522005,
 u'Freq2_harmonics_amplitude_2': 0.0031285480277133165,
 u'Freq2_harmonics_amplitude_3': 0.0037513261381000875,
 u'Freq2_harmonics_rel_phase_0': 0.0,
 u'Freq2_harmonics_rel_phase_1': 1.6636661340599668,
 u'Freq2_harmonics_rel_phase_2': 0.76712330033137777,
 u'Freq2_harmonics_rel_phase_3': 0.63634805297099672,
 u'Freq3_harmonics_amplitude_0': 0.015784199676424832,
 u'Freq3_harmonics_amplitude_1': 0.0024911941274430413,
 u'Freq3_harmonics_amplitude_2': 0.0041985558013

# Timeit

In [7]:
%%timeit -n 10
lscargle = FATSLombScargle(fs)
fats_ls = lscargle.fit(magnitude, time, 6.)

10 loops, best of 3: 1.56 s per loop


In [8]:
%%timeit -n 10
feets_ls = FeetsLombScargle
feets_ls = lscargle.fit(magnitude, time, **params)

10 loops, best of 3: 68.4 ms per loop


# Check Period-Fit

In [9]:
# Create a normal distributed light curve
mag_normal = np.random.normal(size=10000)
time_normal = np.arange(10000)

lscargle.fit(mag_normal, time_normal, **params)

{u'PeriodLS': 0.21549530263056282,
 u'Period_fit': 1.0,
 u'Psi_CS': 0.01796635018421806,
 u'Psi_eta': 1.9822363127179898}

In [10]:
# Create uniform light-curve
mag_uniform = np.random.uniform(size=10000)
time_uniform = np.arange(10000)

lscargle.fit(mag_uniform, time_uniform, **params)

{u'PeriodLS': 0.20958126715026559,
 u'Period_fit': 1.0,
 u'Psi_CS': 0.016809209356035283,
 u'Psi_eta': 1.9741073028293359}

In [11]:
# periodic light-curve

N = 100
time_periodic = np.arange(N)
Period = 10
cov = np.zeros([N, N])
mean = np.zeros(N)
for i in np.arange(N):
    for j in np.arange(N):
        cov[i, j] = np.exp(-(np.sin((np.pi / Period) * (i - j)) ** 2))
mag_periodic = np.random.multivariate_normal(mean, cov)

fasper_kwds = {"autopower_kwds": {"samples_per_peak": 20., "nyquist_factor": 6}}
lscargle.fit(mag_periodic, time_periodic, **params)

{u'PeriodLS': 0.20408864456278986,
 u'Period_fit': 0.0,
 u'Psi_CS': 0.21765105487287698,
 u'Psi_eta': 0.089017191996318051}